In [1]:
import numpy as np

In [9]:
weights = [0.0, 0.1240317450077846, -6.211941063144333, 0.04933903144709126, 0.03810848157715883, 8.132366097133624e-05, -6.018769160916912e-05, -1.251585565299179e-07, 3.484096383229681e-08, 4.1614924993407104e-11, -6.732420176902565e-12]
len(weights)

11

In [ ]:
class GenecticAlgorithm:
    '''
    This class encapsulates all the methods of a genectic algorithm
    '''
    def __init__(self):
        pass
    
    def calculate_fitness(self,weights):
        # returns a list of fitness score for each of the weight vector
        # The fitness will be calculated using the client_mooodle.py
        pass
    
    def crossover(self, parents, offspringSize):
        # generates new children from parents
        # there are various methods of doing it, blend, one-point,two-point, uniform. we need to try these.
        # also keep some of the original parents in the vector so that the new generation performs at least as good the previous one.
        pass
    
    def mutation(self, weights):
        # introduces mutation in the weights
        # randomly change the number by addind some random noise to maybe two-three weights in the weight vector.
        # but the noise has to be scaled to the level of weight to which we are adding it.
        pass
    
    def mating_pool(self, weights, fitness, numberOfParents):
        # selects the best parents for reproducing new generations
        # get the best parents by using the fitness scores, this method will not produce new offspring
        pass
    